# Handling Missing Values and Errors
We all know that the world is far from perfect.  When ever we receive data from some source where humans are involved, the quality of the data is not alway all that great.  Values may be missing in files or database tables, or values may be invalid for the expected data type.  That is why we need a way to handle missing values and errors when working with data.

## Missing Values
In a database table a missing value may be represented as NULL.  In a CSV file the value may simply be empty.  Some programming languages use null (as in SQL and C#) or some other value like "undefined" (as in JavaScript) or "N/A" (as in R) to represent missing values.

In F# you would usually use a special discriminated union: option.  You already saw a glimpse of option in the first example in this guide.  F# also lets you use null but you should avoid null as much as possible.  Using null is [asking for trouble](https://en.wikipedia.org/wiki/Tony_Hoare).

With option you can have two possible values: `Some` and `None` where `None` of course means that the value is missing.  Pattern matching is your friend when using options.  It looks like this.

In [34]:
let someValue = Some 42
let missing = None

let printValue v =
    match v with
    | Some i -> printfn "%d" i
    | None -> printfn "The value is missing."
    
someValue |> printValue
missing |> printValue

42
The value is missing.


As we have seen before, F# automatically derives the type of the `v` argument to the `printValue` function.  The type of `v` in this case is an integer option, written as `int option` if typed out, like so:
```fsharp
    let printValue (v: int option) =
        // ...
```

## Missing Values in CSV files
You can configure the CSV type provider to handle missing values.  By default the CSV type provider will use [`double.NaN`](https://docs.microsoft.com/en-us/dotnet/api/system.double.nan?view=netcore-3.1) (Not a Number) for missing values in double columns and `""` for missing string values.

If you prefer to use `Option` instead, you can configure the CSV provider to use options instead of `double.Nan` and `""`.  Let us try with a new [CSV file](https://raw.githubusercontent.com/t4rzsan/fsharp-for-actuaries/master/PoliciesWithMissingPremiums.csv) where the premium is missing for a couple of row.  These are the contents of the file.

```
PersonId;PolicyNumber;Age;Group;Gender;Premium;TechnicalProvision
1;Pol001;55;R2;m;100;1233,54
1;Pol002;55;R2;m;150;264,23
2;Pol003;32;R3;f;;5465,32
2;Pol004;32;R3;f;300;475,23
3;Pol005;20;R2;m;NA;234
4;Pol006;64;R2;f;167;5343,64```

Notice the premium column is empty in rows 3 and 5.

If you try to open this file with the type provider we created in the chapter on Visualizing Data, you will get an error saying that the Premium is missing.  So let us tell the type provider that it should assume missing values and that it should use `Option`.  We do that by setting `AssumeMissingValues` and `PreferOptionals` to true when we define the type.

In [35]:
#r "nuget: FSharp.Data"
open FSharp.Data

type PoliciesProvider = CsvProvider<Sample="PersonId;PolicyNumber;Age;Group;Gender;Premium;TechnicalProvision", Culture="da-DK", Separators=";", Schema="Premium=decimal option, TechnicalProvision">
let policies = PoliciesProvider.Load("https://raw.githubusercontent.com/t4rzsan/fsharp-for-Actuaries/master/PoliciesWithMissingPremiums.csv").Rows

let printDecimal d =
    match d with
    | Some p -> printfn "%M" p
    | None -> printfn "None"

policies
|> Seq.skip 2
|> Seq.take 2
|> Seq.map (fun p -> p.Premium)
|> Seq.iter printDecimal


None
300
